<a href="https://colab.research.google.com/github/ymuto0302/base1_2021/blob/main/GIS4_2_WDI_Education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# コロプレスマップを用いた World Development Indicators からの教育環境の分析

### モチベーション
経営情報学科のカリキュラムには「国際関係論」や「国際経営論」という科目がある。また，SDGs に基づく社会課題解決という世界的な流れがある。

特に SDGs では貧困，教育，平等性，気候変動等，様々な問題が取り上げられている。

これらのうち，教育と地理情報を絡めてみよう！

---
### 前回の授業
"World Development Indicators" データセットを利用し，指標 access to electorcity からコロプレスマップを構築する事例を説明した後，次の２指標について考察した。
- "People using at least basic drinking water services (% of population)"
- "Total CO2 emissions (thousand metric tons of CO2 excluding Land-Use Change and Forestry)"

### 今回の授業
利用するデータセットは "World Development Indicators (WDI)" である。

https://datacatalog.worldbank.org/dataset/world-development-indicators

このデータから教育に関する指標として次の２指標を抽出し，コロプレスマップの展開および考察を試みる。
- 就学率
- Gender Parity Index


---
## WDI にて提供される指標
1400種超の指標が含まれる。

国名(Country Name / Country Code)×指標(Indicator Name / Indicator Code) ×年度(1960〜2020)に対して「指標の値」が示されている。

- 対象となる国（地域）数：247カ国 ・・・多分・・・
- 指標：1443種類 ・・・WDISeries.csv を参照
- 年度：指標によって収集された件数が異なる

## データ
データはファイル "WDIData.csv" に格納されている。


---
### (準備) Google Drive のマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
### Indicator Code (指標コード) の検索
Indicator Name (指標名)が既知である場合，指標の一覧を格納するファイル WDISeries.csv を読み込み，検索をかければよい。

以下の例では "Access to electoricity" を含む指標を検索している。

(注意) contains() メソッドは大文字／小文字を区別する。

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/BASE/GeoData/WorldDevelopmentIndicators/WDISeries.csv")
df[df['Indicator Name'].str.contains('Access to electricity')]

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
197,EG.ELC.ACCS.RU.ZS,Environment: Energy production & use,"Access to electricity, rural (% of rural popul...",NaN,"Access to electricity, rural is the percentage...",NaN,Annual,NaN,NaN,Weighted average,NaN,NaN,NaN,World Bank Global Electrification Database fro...,Data for access to electricity are collected a...,NaN,NaN,NaN,NaN,CC BY-4.0,NaN
198,EG.ELC.ACCS.UR.ZS,Environment: Energy production & use,"Access to electricity, urban (% of urban popul...",NaN,"Access to electricity, urban is the percentage...",NaN,Annual,NaN,NaN,Weighted average,NaN,NaN,NaN,World Bank Global Electrification Database fro...,Data for access to electricity are collected a...,NaN,NaN,NaN,NaN,CC BY-4.0,NaN
199,EG.ELC.ACCS.ZS,Environment: Energy production & use,Access to electricity (% of population),NaN,Access to electricity is the percentage of pop...,NaN,Annual,NaN,NaN,Weighted average,NaN,NaN,NaN,World Bank Global Electrification Database fro...,Data for access to electricity are collected a...,Maintaining reliable and secure electricity se...,NaN,NaN,NaN,CC BY-4.0,NaN


---
## [テーマ1] 就学率
一般に学校教育は
- primary education: 初等教育（小学校）
- secondary education: 中等教育（中学校・高等学校）
- tertiary education: 高等教育（高等専門学校・大学・大学院）

の３段階に分けられる。もちろん国によって初等教育や中等教育の年限は多少，異なるが， 本授業では細かい点に留意しない。

WDI におけるそれぞれの Indicator Name は以下のとおり：
- 初等教育の就学率・・・School enrollment, primary (% gross)
- 中等教育の就学率・・・School enrollment, secondary (% gross)
- 高等教育の就学率・・・School enrollment, tertiary (% gross)

**(注) gross でなく adjusted net の値も存在するが，国によって集計年度が異なるため，コードがややこしくなる。故に，本授業では（誤差が含まれるものの，傾向を掴むには問題ないとみなし）gross の値を用いる。**

#### school enrollment (% gross) について
school enrollment (% gross) は 100% を超える。
その理由について， unesco.org に分かりやすい説明が掲載されているから，参照してください。

http://uis.unesco.org/en/glossary-term/gross-enrolment-ratio#:~:text=A%20GER%20value%20approaching%20or,indicate%20the%20proportion%20already%20enrolled.


---
## [テーマ2] gender parity index (GPI)
就学率における gender parity index (GPI) とは，「女子の就学率/男子の就学率」で計算される。 すなわち，GPI の値が1を上回ると女子の方が就学率が高く， 逆に1を下回る場合は男子の方が就学率が高いということを示す。

ジェンダーによる不平等性の解消という観点から GPI の値は 1.0 前後にあることが望ましい。

WDI における GPI の Indicator Name は以下のとおり：
- 初等教育のGPI・・・School enrollment, primary (gross), gender parity index
- 中等教育のGPI・・・School enrollment, secondary (gross), gender parity index
- 高等教育のGPI・・・School enrollment, tertiary (gross), gender parity index


GPI の値は国によって調査タイミングや頻度が異なるため，特定の年の値のみを眺めると，以下の問題が生じる。
- アフリカの数値がごっそり抜けている
- 成長著しいインドの数値が抜けている
- 日本を含むアジアの数値がまばらである

そこで，2008, 2010, 2012, 2014, 2016, 2018年の値のうち，最大値を「新たなデータ列 "GPI"」として定義することが望ましい。

以下に，（例えば初等教育の GPI を）変数 df へ取り込んだ場合のコード例を示す。

In [ ]:
# 2008〜2018年の期間において，国によって値の有無があるため，最大値をもって「各国の GPI」とする。
df['GPI'] = df[['2008', '2010', '2012', '2014', '2016', '2018']].max(axis=1)

---
# 課題

## 課題(その1)
ファイル WDISeries.csv を参照し，
初等教育，中等教育，高等教育のそれぞれについて
就学率および　gender parity index の Indicator Code を得よ。

つまり，**「初等，中等，高等」×「就学率，GPI」の計６指標を対象とする。**

#### Indicator Name の一覧
- 初等教育の就学率・・・School enrollment, primary (% gross)
- 中等教育の就学率・・・School enrollment, secondary (% gross)
- 高等教育の就学率・・・School enrollment, tertiary (% gross)
- 初等教育のGPI・・・School enrollment, primary (gross), gender parity index
- 中等教育のGPI・・・School enrollment, secondary (gross), gender parity index
- 高等教育のGPI・・・School enrollment, tertiary (gross), gender parity index


## 課題(その2)
初等教育，中等教育，高等教育のそれぞれについて
就学率および　gender parity index の値をコロプレスマップへ投影し，経済的・政治的観点から考察せよ。

**(注) GPI はかなり歯抜けになっている → 前述したとおり，複数の年から最大値をとる，といった前処理が必要である。**
